<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/Financial_Market_Analysis_with_MVO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install cvxpy
import cvxpy as cp
import numpy as np
from datetime import datetime

# Expected returns
mu = np.array([0.10, 0.15, 0.20])

# Covariance matrix
Sigma = np.array([
    [0.025, 0.003, 0.002],
    [0.003, 0.045, 0.005],
    [0.002, 0.005, 0.07]
])

# Desired portfolio return
mu_p = 0.15

# Number of assets
n = len(mu)

# Optimization Variables
w = cp.Variable(n)

# Portfolio variance
portfolio_variance = cp.quad_form(w, Sigma)

# Objective Function: Minimize portfolio variance
objective = cp.Minimize(portfolio_variance)

# Constraints
constraints = [cp.sum(w) == 1, w.T @ mu == mu_p, w >= 0]

# Problem
problem = cp.Problem(objective, constraints)

# Solve the problem
problem.solve()

# Optimal weights
print("Optimal Portfolio Weights:")
for i, weight in enumerate(w.value):
    print(f"Asset {chr(65+i)}: {weight:.2f}")

print(f"\nMinimum Portfolio Variance: {portfolio_variance.value:.4f}")

###########
import yfinance as yf
import numpy as np
import pandas as pd
import cvxpy as cp

# Updated tickers (replace 'FB' with 'META' or remove it if not applicable)
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']  # Assume 'META' is the updated ticker for Facebook

# Use today's date as the end date
end_date = datetime.today().strftime('%Y-%m-%d')

# Fetch historical stock data
data = yf.download(tickers, start="2020-01-01", end=end_date)['Adj Close']

# Handle possible missing data
data = data.dropna()

# Calculate daily returns and drop any NaN values that may result from the data
returns = data.pct_change().dropna()

# Check if the returns DataFrame is empty
if returns.empty:
    raise ValueError("No data available for the specified tickers and date range. Please check your tickers and dates.")

# Calculate expected returns and the covariance matrix
expected_returns = returns.mean()
cov_matrix = returns.cov()

# Ensure the covariance matrix is not empty or singular
if cov_matrix.isnull().values.any() or np.linalg.matrix_rank(cov_matrix.values) < len(tickers):
    raise ValueError("Covariance matrix issue detected. Please check the data quality.")

# Number of assets
n = len(expected_returns)

# Optimization Variables
w = cp.Variable(n)

# Portfolio variance
portfolio_variance = cp.quad_form(w, cov_matrix.values)

# Objective Function: Minimize portfolio variance
objective = cp.Minimize(portfolio_variance)

# Constraints: Weights sum to 1
constraints = [cp.sum(w) == 1, w >= 0]

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Display the optimal portfolio weights
print("Optimal Portfolio Weights:")
for ticker, weight in zip(expected_returns.index, w.value):
    print(f"{ticker}: {weight:.2%}")

#######

# Install necessary packages
import yfinance as yf
import cvxpy as cp
import numpy as np
import pandas as pd

# Fetch data
tickers = ['AAPL', 'MSFT', 'GOOGL']
data = yf.download(tickers, start="2020-01-01", end=end_date)['Adj Close']

# Ensure no missing data
if data.isnull().values.any():
    print("Warning: Missing data points detected. Filling forward.")
    data.fillna(method='ffill', inplace=True)

returns = data.pct_change().dropna()

mu = returns.mean().values
Sigma = returns.cov().values

# Adjust mu_p if needed
mu_p = np.mean(mu)  # Example adjustment based on mean of expected returns

n = len(mu)
w = cp.Variable(n)
portfolio_variance = cp.quad_form(w, Sigma)

objective = cp.Minimize(portfolio_variance)
constraints = [cp.sum(w) == 1, w @ mu == mu_p, w >= 0]

problem = cp.Problem(objective, constraints)

# Try solving with a different solver if the default doesn't work
try:
    problem.solve()
except Exception as e:
    print(f"Default solver failed: {e}. Trying with another solver...")
    problem.solve(solver=cp.SCS)

if w.value is not None and not np.isnan(portfolio_variance.value):
    print("Optimal Portfolio Weights:")
    for ticker, weight in zip(tickers, w.value):
        print(f"{ticker}: {weight:.2%}")
    print(f"\nMinimum Portfolio Variance: {portfolio_variance.value:.4f}")
else:
    print("Problem not solved. Check problem setup and data.")


Optimal Portfolio Weights:
Asset A: 0.33
Asset B: 0.34
Asset C: 0.33

Minimum Portfolio Variance: 0.0178


[*********************100%%**********************]  5 of 5 completed
[*********************100%%**********************]  3 of 3 completed


Optimal Portfolio Weights:
AAPL: 31.19%
AMZN: 13.85%
GOOGL: 26.94%
META: 0.00%
MSFT: 28.02%
Optimal Portfolio Weights:
AAPL: 34.39%
MSFT: 32.80%
GOOGL: 32.81%

Minimum Portfolio Variance: 0.0004


The provided Python snippet and its application in finance indeed illustrate a practical use of Transportation-Cost Inequalities (TCI) and Relative Entropy (Kullback-Leibler Divergence) in portfolio risk management, directly echoing the theoretical principles introduced by mathematicians and economists in the field of modern portfolio theory and optimization.

In finance, particularly within the scope of portfolio management and optimization, the concepts of TCI and Relative Entropy serve as quantitative measures for assessing and managing the "distance" or discrepancy between an actual portfolio and a target or theoretical portfolio. This approach offers a structured method to quantify the effort, cost, or risk associated with adjusting the portfolio to align with a predefined optimal or desired state, thus facilitating more informed decision-making based on mathematical principles.

The transportation cost, calculated as the squared difference between the current and target allocations for each asset, reflects the quadratic increase in effort or cost associated with larger adjustments. This models real-world scenarios where significant portfolio adjustments entail higher transaction costs or potentially introduce greater market impact costs.

Relative Entropy, or Kullback-Leibler Divergence, complements this by measuring the divergence of the current portfolio from the target in informational terms. It offers insights into the inefficiency of the current portfolio compared to the target, providing a basis for understanding the informational cost of maintaining the current portfolio instead of the optimal or target portfolio.

Together, these measures equip financial managers and analysts with powerful tools for risk assessment and portfolio optimization, grounded in the work of Harry Markowitz and extended by various researchers who have contributed to the development of modern portfolio theory and its applications. Through the application of these theoretical principles to real-world data, as exemplified in your Python snippet, investors and portfolio managers can make data-driven decisions to optimize portfolio performance while managing risk in a quantifiable manner.